In [1]:
# loading EASYMORE
from easymore.easymore import easymore

# initializing EASYMORE object
esmr = easymore()



# execute EASYMORE
esmr.dataframe_to_netcdf('../data/station_data/station_data.csv',
                         'Date_time',
                         '../data/station_data/station_data.nc',
                            'ss',
                            'sss',
                            'ssss',
                            time_step = 'seconds', # minutes, hours, or days
                            calendar = 'standard',
                            station_info_name = '../data/station_data/station_info.csv')



EASYMORE version 0.0.2 is initiated.


In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/station_data/station_info.csv')
df = df.set_index('Unnamed: 0')
df.index.names = [None]
print(df)
ID = np.array(df.loc['ID'])
lat = np.array(df.loc['lat'])
lon = np.array(df.loc['lon'])
print(ID, lat, lon)

     Station_1  Station_2  Station_3
ID           1          3        100
lat          3          4          5
lon         -2         -7         -5
[  1   3 100] [3 4 5] [-2 -7 -5]


In [19]:
df = pd.read_csv('../data/station_data/station_data.csv')
print(df)


            Date_time  Station_1  Station_2  Station_3
0  2001-01-01 1:00:00          2        3.0          3
1  2001-01-02 1:00:00          3        2.0          2
2  2001-01-05 4:00:00          4        3.6          1
3  2001-02-01 1:00:00          1    -9999.0          2
4  2001-03-01 1:00:00          2        3.0          3


In [2]:
import xarray as xr
ds = xr.open_dataset('../data/station_data/station_data.nc')
print(ds)


<xarray.Dataset>
Dimensions:      (time: 5, n: 3)
Coordinates:
  * time         (time) datetime64[ns] 2001-01-01T01:00:00 ... 2001-03-01T01:...
Dimensions without coordinates: n
Data variables:
    ss           (time, n) float64 ...
    column_name  (n) object ...
    ID           (n) int32 ...
    latitude     (n) float64 ...
    longitude    (n) float64 ...
    Station_ID   (n) object ...
Attributes:
    Conventions:  CF-1.6
    Author:       The data were written by 
    License:      
    History:      Created Sun Jan 30 22:21:49 2022
    Source:       Case: case_temp; remapped by script from library of Shervan...
